# 0. 기본 정보
- 코드 작성자: 여서연
- 코드 작성일: 2024-10-06
- 코드 작성 목적: 지정한 투자 전략의 백테스팅 구현 실습

# 1. 기초 설정

## 사용 라이브러리

In [1]:
# 백테스팅
import backtrader as bt
import yfinance as yf

In [2]:
# 파라미터 최적화
from functools import partial
import pandas as pd

## 기타 설정

In [3]:
# 소수점 아래 2자리까지 표시
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# 한국어 및 한국 지역 형식 설정
import locale

locale.setlocale(locale.LC_ALL, 'ko_KR')

'ko_KR'

In [5]:
# 모든 경고 무시
import warnings

warnings.filterwarnings('ignore')

# 2. 전략 정의

In [6]:
# 볼린저 밴드 계산 지표
class BollingerBands(bt.Indicator):
    lines = ('mid', 'top', 'bot')  # 중간 밴드(이동평균선), 상단 밴드, 하단 밴드
    params = (('period', 30), ('devfactor', 3))  # 이동 평균 기간과 표준 편차 배수

    def __init__(self):
        self.lines.mid = bt.indicators.SMA(self.data.close, period=self.params.period)  # 단순 이동 평균(SMA)
        stddev = bt.indicators.StandardDeviation(self.data.close, period=self.params.period)  # 표준 편차
        self.lines.top = self.lines.mid + (self.params.devfactor * stddev)  # 상단 밴드 = SMA + (표준편차 * 배수)
        self.lines.bot = self.lines.mid - (self.params.devfactor * stddev)  # 하단 밴드 = SMA - (표준편차 * 배수)

In [7]:
# 평균 회귀 전략
class MeanReversionStrategy(bt.Strategy):
    params = (
        ('period', 30),           # 볼린저 밴드 기준이 되는 이동 평균 기간
        ('devfactor', 3),         # 표준 편차의 배수
        ('stop_loss', 0.98),      # 손절매 비율 (2% 손실 시 손절)
        ('take_profit', 1.05),    # 목표 수익 비율 (5% 수익 시 이익 실현)
        ('notice', True),         # 거래 시 알림 출력 여부
    )

    def __init__(self):
        # BollingerBands 지표를 불러와 사용 (self.data를 직접 전달)
        self.bollinger = BollingerBands(self.data, period=self.params.period, devfactor=self.params.devfactor)
        self.buy_price = None  # 매수 가격 기록

    def next(self):
        close = self.data.close[0]  # 현재 종가
        cash = self.broker.getcash()  # 현재 현금 잔고
        size = int(cash / (close * 1.002))  # 매수 가능한 최대 주식 수(수수료 및 슬리피지를 고려한 버퍼 추가)
        notice = self.params.notice  # 알림 여부

        # 현재 종가와 볼린저 밴드 값 출력
        if notice:
            print(f"Close: {close:.2f}, Lower Band: {self.bollinger.bot[0]:.2f}, Upper Band: {self.bollinger.top[0]:.2f}")

        # 매수: 종가가 하단 밴드보다 낮을 때
        if not self.position:
            if close < self.bollinger.bot[0]:
                if size > 0:
                    if notice:
                        print(f"Buying {size} shares at {close} (Below Bollinger Band)")
                    self.buy(size=size)
                    self.buy_price = close  # 매수 가격 기록
                else:
                    if notice:
                        print("Not enough cash to buy")
        else:
            # 매도: 종가가 상단 밴드보다 높을 때
            if close > self.bollinger.top[0]:
                if notice:
                    print(f"Selling {self.position.size} shares at {close} (Above Bollinger Band)")
                self.close()

            # 손절: 현재 종가가 매수가 대비 2% 이상 하락했을 때
            elif close < self.buy_price * self.params.stop_loss:
                if notice:
                    print(f"Stop loss triggered: Selling {self.position.size} shares at {close}")
                self.close()

            # 목표 수익 실현: 현재 종가가 매수가 대비 5% 이상 상승했을 때
            elif close > self.buy_price * self.params.take_profit:
                if notice:
                    print(f"Take profit triggered: Selling {self.position.size} shares at {close}")
                self.close()

    def notify_order(self, order):
        notice = self.params.notice
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price  # 매수 가격 저장
                action = 'Buy'
            elif order.issell():
                profit = (order.executed.price - self.buy_price) * order.executed.size
                if notice:
                    print(f'Profit from sale: {profit:.2f}')
                action = 'Sell'

            stock_price = self.data.close[0]
            cash = self.broker.getcash()
            value = self.broker.getvalue()

            if notice:
                print(f'{action} order completed: price[{stock_price:.2f}] cash[{cash:.2f}] value[{value:.2f}]')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if notice:
                print(f'Order {order.ref} was {order.getstatusname()}')

# 3. 시뮬레이션

In [8]:
# 데이터 다운로드
# yahoo = bt.feeds.PandasData(dataname=yf.download("TQQQ", start="2018-01-01", end="2023-12-31"))
data = bt.feeds.PandasData(dataname=yf.download("TQQQ", period='5d', interval='1m'))
# ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']")

[*********************100%***********************]  1 of 1 completed


In [9]:
cerebro = bt.Cerebro()  # Cerebro 엔진 생성
cerebro.broker.setcash(100_000_000)  # 초기 자본 설정
cerebro.broker.setcommission(0.002)  # 거래 수수료 설정
cerebro.adddata(data)  # 데이터 추가
cerebro.addstrategy(MeanReversionStrategy)  # 전략 추가

start_value = cerebro.broker.getvalue()  # 백테스트 시작 시 자산 값 저장
cerebro.run()  # 백테스트 실행
final_value = cerebro.broker.getvalue()  # 백테스트 종료 후 자산 값 저장

Close: 71.73, Lower Band: 71.52, Upper Band: 72.62
Close: 71.72, Lower Band: 71.62, Upper Band: 72.54
Close: 71.81, Lower Band: 71.60, Upper Band: 72.55
Close: 71.85, Lower Band: 71.59, Upper Band: 72.56
Close: 71.72, Lower Band: 71.55, Upper Band: 72.59
Close: 71.58, Lower Band: 71.47, Upper Band: 72.63
Close: 71.64, Lower Band: 71.42, Upper Band: 72.66
Close: 71.62, Lower Band: 71.37, Upper Band: 72.67
Close: 71.55, Lower Band: 71.31, Upper Band: 72.68
Close: 71.48, Lower Band: 71.24, Upper Band: 72.71
Close: 71.56, Lower Band: 71.20, Upper Band: 72.72
Close: 71.59, Lower Band: 71.16, Upper Band: 72.72
Close: 71.70, Lower Band: 71.14, Upper Band: 72.71
Close: 71.59, Lower Band: 71.11, Upper Band: 72.71
Close: 71.52, Lower Band: 71.07, Upper Band: 72.72
Close: 71.35, Lower Band: 71.01, Upper Band: 72.72
Close: 71.25, Lower Band: 70.93, Upper Band: 72.74
Close: 71.30, Lower Band: 70.88, Upper Band: 72.73
Close: 71.21, Lower Band: 70.82, Upper Band: 72.72
Close: 71.27, Lower Band: 70.79

In [10]:
# 자산 변화 출력
print('* start value : %s $' % locale.format_string('%d', start_value, grouping=True))
print('* final value : %s $' % locale.format_string('%d', final_value, grouping=True))
print('* earning rate : %.2f %%' % ((final_value - start_value) / start_value * 100.0))

# 백테스트 결과 차트 출력
cerebro.plot()

* start value : 100,000,000 $
* final value : 106,564,266 $
* earning rate : 6.56 %


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]

# 4. 파라미터 비교

## 함수 설정

In [11]:
def Simulate(strategy_cls, data, **kwargs):
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100_000_000)
    cerebro.broker.setcommission(0.002)
    cerebro.adddata(data)
    cerebro.addstrategy(partial(strategy_cls, **kwargs))
    start_value = cerebro.broker.getvalue()
    cerebro.run()
    final_value = cerebro.broker.getvalue()
    earning_rate = ((final_value - start_value) / start_value * 100.0)

    return start_value, final_value, earning_rate

## 기존 데이터 사용

In [12]:
# periods와 devfactors 설정
periods = [i for i in range(10, 40, 10)]
devfactors = [i for i in range(1, 4)]  # 표준편차 배수 범위

In [13]:
# 시뮬레이션 수행
results = []
for period in periods:
    for devfactor in devfactors:
        start_value, final_value, earning_rate = Simulate(
            MeanReversionStrategy,
            data,
            period=period,
            devfactor=devfactor,
            notice=False
        )
        results.append({
            'Period': period,
            'Dev Factor': devfactor,
            'Start Value': start_value,
            'Final Value': final_value,
            'Earning Rate (%)': earning_rate
        })

In [14]:
# 결과 데이터프레임 생성성
df = pd.DataFrame(results)
df

,Period,Dev Factor,Start Value,Final Value,Earning Rate (%)
0,10,1,100000000,72244833.59,-27.76
1,10,2,100000000,85473305.48,-14.53
2,10,3,100000000,100000000.00,0.00
3,20,1,100000000,80714399.44,-19.29
4,20,2,100000000,88198366.36,-11.80
5,20,3,100000000,105490827.43,5.49
6,30,1,100000000,87413142.13,-12.59
7,30,2,100000000,94751293.48,-5.25
8,30,3,100000000,106564266.73,6.56


볼린저밴드의 범위가 넓을 수록 수익률이 좋다. 다른 데이터에서도 그럴까?

## 다른 데이터 사용

In [15]:
data2 = bt.feeds.PandasData(dataname=yf.download("SQQQ", period='5d', interval='1m'))

[*********************100%***********************]  1 of 1 completed


In [16]:
periods = [i for i in range(10, 40, 10)]
devfactors = [i for i in range(1, 4)]  # 표준편차 배수 범위

results = []
for period in periods:
    for devfactor in devfactors:
        start_value, final_value, earning_rate = Simulate(
            MeanReversionStrategy,
            data2,
            period=period,
            devfactor=devfactor,
            notice=False
        )
        results.append({
            'Period': period,
            'Dev Factor': devfactor,
            'Start Value': start_value,
            'Final Value': final_value,
            'Earning Rate (%)': earning_rate
        })

# Create a DataFrame to display the results
df2 = pd.DataFrame(results)
df2

,Period,Dev Factor,Start Value,Final Value,Earning Rate (%)
0,10,1,100000000,74215473.16,-25.78
1,10,2,100000000,91808159.17,-8.19
2,10,3,100000000,100000000.00,0.00
3,20,1,100000000,81017032.96,-18.98
4,20,2,100000000,91046947.39,-8.95
5,20,3,100000000,99740887.38,-0.26
6,30,1,100000000,86874611.19,-13.13
7,30,2,100000000,93924294.93,-6.08
8,30,3,100000000,101405909.71,1.41
